In [2]:
from kendall_tau import compute_kendall_tau

taus = compute_kendall_tau("data/processed/train_processed.csv", "data/processed/train_reordered_pairs.csv")

print("Average Kendall tau:", sum(taus)/len(taus))

Average Kendall tau: 0.35985454545453716
